#  Analyses of UT cells and after exposure to anti-fungal drugs (FCZ day 2 & 3, CSP day 2, and RAPA day 2)


## 0. Loading the libraries

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from umap import UMAP

In [3]:
import scanpy as sc
import numpy as np
import scipy as sp
import pandas as pd

import meld
import magic

In [4]:

%matplotlib inline
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sb
sb.set_context('notebook')
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
plt.rcParams['figure.figsize']=(8,8) #rescale figures
plt.rcParams.update({'figure.max_open_warning': 0})


from gprofiler import gprofiler

#import sys


import scprep

import graphtools as gt


In [5]:
import rpy2
print(rpy2.__version__)

3.4.2


In [6]:
# Ignore R warning messages
# Note: this can be commented out to get more verbose R output
import rpy2.rinterface_lib.callbacks
import logging
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

# Automatically convert rpy2 outputs to pandas dataframes
import rpy2.robjects as ro

from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects import numpy2ri
numpy2ri.activate()

import anndata2ri
anndata2ri.activate()

%reload_ext rpy2.ipython

sc.settings.verbosity = 3
#sc.set_figure_params(dpi=200, dpi_save=300)
sc.logging.print_versions()

import scvi
import anndata
import random

-----
anndata     0.7.6
scanpy      1.9.2
-----
PIL                 8.3.1
anndata2ri          0.0.0
autoreload          NA
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
certifi             2022.09.24
cffi                1.14.6
chardet             4.0.0
charset_normalizer  2.0.0
colorama            0.4.4
concurrent          NA
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
decorator           4.4.2
defusedxml          0.7.1
deprecated          1.2.12
dunamai             1.6.0
encodings           NA
future              0.18.2
genericpath         NA
get_version         3.5
google              NA
gprofiler           1.0.0
graphtools          1.5.2
h5py                3.3.0
idna                3.2
igraph              0.9.6
ipykernel           6.1.0
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib              1.0.1
kiwisolver          1.3.1
leidenalg  

<a id="Reading"></a>

## 1. Reading in the data

In [7]:

adata = anndata.read_h5ad('../../data/data03-scvi/finalall-adata.h5ad')

In [8]:
# Annotate the data sets
print(adata.obs['orig.ident'].value_counts())
print('')
print(adata.obs['nruns'].value_counts())
print('')

UT_0_1_FALSE      2736
FCZ_6_3_FALSE     2483
RAPA_2_2_FALSE    2460
FCZ_2_2_FALSE     2444
CSP_2_1_FALSE     2075
UT_0_3_FALSE      2050
FCZ_3_2_FALSE     1616
FCZ_2_1_FALSE      948
FCZ_3_3_FALSE      908
FCZ_3_1_FALSE      858
UT_0_2_FALSE       276
Name: orig.ident, dtype: int64

3.0    12237
2.0     6617
Name: nruns, dtype: int64



In [8]:
adata = adata[(adata.obs["drug_day"].isin(["UT_0","FCZ_2", "FCZ_3", "CSP_2", "RAPA_2"]))]

In [9]:
print(adata.obs['orig.ident'].value_counts())
print('')

UT_0_1_FALSE      2736
RAPA_2_2_FALSE    2460
FCZ_2_2_FALSE     2444
CSP_2_1_FALSE     2075
UT_0_3_FALSE      2050
FCZ_3_2_FALSE     1616
FCZ_2_1_FALSE      948
FCZ_3_3_FALSE      908
FCZ_3_1_FALSE      858
UT_0_2_FALSE       276
Name: orig.ident, dtype: int64



## 2. MAGIC imputation

Count data are first notmalized for library size and square-root transformed. We then aply MAGIC (Markov Affinity-based Graph Imputation of Cells), an algorithm for denoising and imputation of single cells applied to single-cell RNA sequencing data, as described in Van Dijk D et al. (2018), Recovering Gene Interactions from Single-Cell Data Using Data Diffusion, Cell. [https://www.cell.com/cell/abstract/S0092-8674(18)30724-4]

In [ ]:
import magic
data_libnorm, libsize = scprep.normalize.library_size_normalize(adata.X, return_library_size=True)
data_sqrt = scprep.transform.sqrt(data_libnorm)
data_magic = magic.MAGIC().fit_transform(data_sqrt)

Calculating MAGIC...
  Running MAGIC on 16371 cells and 5277 genes.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 2.14 seconds.
    Calculating KNN search...
    Calculated KNN search in 32.91 seconds.
    Calculating affinities...


In [ ]:
adata.layers["magic"]=data_magic

In [ ]:
data_magic
gene_list = adata.var.index.values
dge = pd.DataFrame(data_magic, columns = gene_list)

# dge.to_csv("../../data/data03-scvi/utfczcsprapa-adjLogcounts_magic.csv")

## 3. scVI

In [14]:
adata.layers["counts"] = adata.X.copy() # preserve counts
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata # freeze the state in `.raw`

normalizing counts per cell
    finished (0:00:00)


In [19]:
adata
adata.write('utfczcsprapa_processed.h5ad')

In [ ]:
adata = anndata.read('utfczcsprapa_processed.h5ad')
adata

In [ ]:
all_columns = adata.obs_names.tolist()
all_columns

leiden_scVI_vectors = []

In [ ]:
random.shuffle(all_columns)
sample_columns = all_columns[:int(0.95 * len(all_columns))]
sample_columns

In [17]:
random.shuffle(all_columns)
sample_columns = all_columns[:int(0.95 * len(all_columns))]
sample_columns

['UT_0_3_FALSE_GACCGGAGTCCA',
 'CSP_2_1_FALSE_GCAAGTCGCGTG',
 'UT_0_1_FALSE_GATGCTCCTATA',
 'RAPA_2_2_FALSE_TCCCGGCCGCGA',
 'UT_0_3_FALSE_GTGTTGTCGTTA',
 'RAPA_2_2_FALSE_GAACGTGCCGAA',
 'CSP_2_1_FALSE_CGCCATCTTCAA',
 'UT_0_1_FALSE_CAATTCACGGAC',
 'UT_0_1_FALSE_TGTCACCGCGAG',
 'UT_0_1_FALSE_AATGCTTCATTA',
 'CSP_2_1_FALSE_CAATCAAGATGT',
 'FCZ_2_1_FALSE_CCCCAACTTGAC',
 'UT_0_1_FALSE_TGTTCGTTATTC',
 'FCZ_3_2_FALSE_CATCCTCTGAGA',
 'UT_0_1_FALSE_TATAATCTAACG',
 'FCZ_2_1_FALSE_CACGACGATTTG',
 'FCZ_3_1_FALSE_GCTTATCTCCTA',
 'FCZ_3_3_FALSE_CTGAAAATTGGT',
 'CSP_2_1_FALSE_AATCTAGGGGAG',
 'UT_0_3_FALSE_TTAATTAACGGC',
 'FCZ_3_2_FALSE_ATAGGCCCAACT',
 'RAPA_2_2_FALSE_AACAACGATATG',
 'FCZ_2_2_FALSE_TTATTGCAACTA',
 'CSP_2_1_FALSE_TCATGAGTTCTT',
 'UT_0_1_FALSE_CATGCTATGTAT',
 'FCZ_3_2_FALSE_AGCATGTAAGTT',
 'FCZ_2_2_FALSE_CAACGTTGTTTC',
 'FCZ_3_2_FALSE_GCCGTAGCGTCA',
 'CSP_2_1_FALSE_GGTCCATGTCTC',
 'FCZ_2_2_FALSE_ACCTGATTGGGA',
 'UT_0_1_FALSE_TGCCCCGTTGCG',
 'UT_0_1_FALSE_GCTCTTTAAATA',
 'UT_0_1_FALSE_GT

In [19]:
len(sample_columns)

15552

In [ ]:
tmp_adata = adata[(adata.obs_names.isin(sample_columns))]

tmp_adata

scVI is a hierarchical Bayesian model for single-cell RNA sequencing data with conditional distributions parametrized by neural networks. Working as a hybrid between a neural network and a bayesian network, scVI performs data harmonization. VAE refers to variational auto-encoders for single-cell gene expression data. scVI is similar to VAE as it tries to bring a more suitable structure to the latent space. While VAE allows users to make observations in a semi-supervised fashion, scVI is easier to train and specific cell-type labels for the dataset are not required in the pure unsupervised case.

Define the scVI model
First, we define the model and its hyperparameters:

n_hidden: number of units in the hidden layer = 128
n_latent: number of dimensions in the shared latent space = 30 (how many dimensions in z)
n_layers: number of layers in the neural network
dispersion: 'gene': each gene has its own dispersion parameter; 'gene-batch': each gene in each batch has its own dispersion parameter Then, we define a trainer using the model and the dataset to train it with in the unsupervised setting, train_size=1.0 and all cells are used for training

Below we provide then use a wrapper function designed to compute scVI's latent representation of the non-normalized data. Specifically, we train scVI's VAE, compute and store the latent representation then return the posterior which will later be used for further inference.

In [10]:
adata = anndata.read('utfczcsprapa_processed.h5ad')


def process_random_columns(adata, sample_size=0.95):
    all_columns = adata.obs_names.tolist()
    
    # leiden_scVI_vectors = {} # list to store the resulting leiden_scVI vectors
          
    random.shuffle(all_columns)
        
    # Select a random sample of the columns
        
    sample_columns = all_columns[:int(sample_size * len(all_columns))]
               
    tmp_adata = adata[(adata.obs_names.isin(sample_columns))].copy()
        
    scvi.data.setup_anndata(
                tmp_adata,
                layer="counts",
                batch_key='batch',
                continuous_covariate_keys=["log_counts"])
            
    model = scvi.model.SCVI(tmp_adata)
            
    model.train()
            
    tmp_adata.layers["scvi_normalized"] = model.get_normalized_expression(
                library_size=10e4
            )
            
    latent = model.get_latent_representation()
            
    tmp_adata.obsm["X_scVI"] = latent
            
    sc.pp.neighbors(tmp_adata, use_rep="X_scVI")
            
    # sc.tl.umap(tmp_adata, min_dist=0.3)
            
    sc.tl.leiden(tmp_adata, key_added="leiden_scVI", resolution=0.4)
    
    leiden_scVI_vectors = pd.DataFrame({'cellid': tmp_adata.obs_names.tolist(), 'leiden_scvi': tmp_adata.obs['leiden_scVI']})
    
    return leiden_scVI_vectors


In [ ]:
results = []
for i in range(100):
    result = process_random_columns(adata)
    results.append(result)
    
# Merge the DataFrames
df_merged = results[0]

for df in results[1:]:
    df_merged = df_merged.merge(df, on='cellid', how='outer')

#df_merged.to_csv('scvi_iteration.csv', index=False)

INFO     Using batches from adata.obs["batch"]                                               
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["counts"]                                              
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 15552 cells, 5277 vars, 3 batches,
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 1 extra
         continuous covariates.                                                              
INFO     Please do not further modify adata until model is trained.                          


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]


Epoch 400/400: 100%|███████████████████████████████████████████████████████████████| 400/400 [08:11<00:00,  1.23s/it, loss=384, v_num=1]
computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:01)
running Leiden clustering
    finished: found 27 clusters and added
    'leiden_scVI', the cluster labels (adata.obs, categorical) (0:00:02)
INFO     Using batches from adata.obs["batch"]                                               
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["counts"]                                              
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 15552 cells, 5277 vars, 3 batches,
         1 labels, and 0 proteins. Also registered 0 extra 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]


Epoch 400/400: 100%|███████████████████████████████████████████████████████████████| 400/400 [08:22<00:00,  1.26s/it, loss=391, v_num=1]
computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:01)
running Leiden clustering
    finished: found 24 clusters and added
    'leiden_scVI', the cluster labels (adata.obs, categorical) (0:00:02)
INFO     Using batches from adata.obs["batch"]                                               
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["counts"]                                              
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 15552 cells, 5277 vars, 3 batches,
         1 labels, and 0 proteins. Also registered 0 extra 

...

Epoch 400/400: 100%|███████████████████████████████████████████████████████████████| 400/400 [07:54<00:00,  1.19s/it, loss=396, v_num=1]
computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:01)
running Leiden clustering
    finished: found 22 clusters and added
    'leiden_scVI', the cluster labels (adata.obs, categorical) (0:00:03)
INFO     Using batches from adata.obs["batch"]                                               
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["counts"]                                              
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 15552 cells, 5277 vars, 3 batches,
         1 labels, and 0 proteins. Also registered 0 extra 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]


Epoch 400/400: 100%|███████████████████████████████████████████████████████████████| 400/400 [08:25<00:00,  1.26s/it, loss=388, v_num=1]
computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:01)
running Leiden clustering
    finished: found 26 clusters and added
    'leiden_scVI', the cluster labels (adata.obs, categorical) (0:00:01)
INFO     Using batches from adata.obs["batch"]                                               
INFO     No label_key inputted, assuming all cells have same label                           
INFO     Using data from adata.layers["counts"]                                              
INFO     Computing library size prior per batch                                              
INFO     Successfully registered anndata object containing 15552 cells, 5277 vars, 3 batches,
         1 labels, and 0 proteins. Also registered 0 extra 

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7,8,9]


Epoch 257/400:  64%|████████████████████████████████████████▍                      | 257/400 [04:58<02:41,  1.13s/it, loss=387, v_num=1]